### Read data 

In [ ]:
# import necessary functions
from helpers.readers import Reader
from helpers.visualization import Visualization
from sklearn.cluster import DBSCAN 
import plotly.express as px
import open3d as o3d 
import numpy as np 
import pandas as pd 

# read data from 
reader = Reader('/home/dim26fa/data/suresim_simulations/')
reader.get_folders('tri')

In [ ]:
reader.get_files_from_folder(-1)
reader.filter('.txt')
reader.set_file(reader.files[1])
reader.folder

In [ ]:
data = reader.read_txt(columns = 4)
data_xyz = reader.extract_xyz(column_names=[0,1,2])

In [ ]:
len(data_xyz)

In [ ]:
viz = Visualization(data_xyz)
viz.get_3d_scatter()

In [ ]:
o3d_pc = o3d.io.read_point_cloud(reader.folder + '/' + reader.file, format = 'xyz')
downpc = o3d_pc.voxel_down_sample(voxel_size = 100)
type(downpc)
downpc_array = pd.DataFrame(np.asarray(downpc.points), columns = ['x','y','z'])

In [ ]:
len(downpc_array)

In [ ]:
viz = Visualization(downpc_array)
viz.get_3d_scatter()

### Find optimal eps for DBSCAN

In [ ]:
from sklearn.neighbors import NearestNeighbors
import numpy as np
import matplotlib.pyplot as plt 

nbrs = NearestNeighbors(n_neighbors=20).fit(data_xyz)
distances, indices = nbrs.kneighbors(data_xyz)
distances = distances[:,2]
distances = np.sort(distances, axis=0)
plt.plot(distances)

### Denoise point cloud

In [ ]:
pc = DBSCAN(eps = 50, min_samples = 6).fit(data_xyz)
print (pc)
type(pc)

In [ ]:
downpc_array[pc.labels_ == -1]

In [ ]:
fig = px.scatter_3d(x = data_xyz['x'], y = data_xyz['y'], z = data_xyz['z'], color = pc.labels_)
fig.update_traces(marker=dict(size=2))

In [ ]:
data_without_outliers = data_xyz.drop(data_xyz[pc.labels_==-1].index)

In [ ]:
fig = px.scatter_3d(x = data_without_outliers['x'], y = data_without_outliers['y'], z = data_without_outliers
              ['z'], color = pc.labels_[pc.labels_ != -1])
fig.update_traces(marker=dict(size=2))

### Construct graphs 

In [ ]:
from sklearn.neighbors import KNeighborsTransformer
import networkx as nx
from pyvis.network import Network
import matplotlib.pyplot as plt

In [ ]:
transformer = KNeighborsTransformer(n_neighbors = 6, algorithm='ball_tree')
transformer.fit_transform(data_without_outliers)

In [ ]:
graph = transformer.kneighbors_graph()
nx_graph = nx.from_numpy_array(graph.toarray())
len(nx_graph.edges)

In [ ]:
plt.imshow(graph.toarray(), cmap='jet')

In [ ]:
net = Network(notebook = True)
net.show_buttons(filter_=['physics'])
net.from_nx(nx_graph)
net.show('graph.html')

In [ ]:
A=list(nx_graph.subgraph(c).copy() for c in nx.connected_components(nx_graph))

In [ ]:
net = Network(notebook = True)
net.show_buttons(filter_=['physics'])
net.from_nx(A[1])
net.show('graph.html')